<p style="color:#153462; 
          font-weight: bold; 
          font-size: 30px; 
          font-family: Gill Sans, sans-serif; 
          text-align: center;">
          Building an ANN</p>

### Importing Required Modules

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import (LabelEncoder,
                                   OneHotEncoder,
                                   StandardScaler)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
tf.__version__

'2.12.0'

### Data Processing

In [3]:
dataset = pd.read_csv("data/Churn_Modelling.csv")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = dataset.loc[:, "CreditScore":"EstimatedSalary"].values
y = dataset.iloc[:, -1].values

In [5]:
print(X[0])

[619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]


In [6]:
print(y)

[1 0 1 ... 1 1 0]


#### Encoding the data

<p style="text-align: justify; text-justify: inter-word;">
   <font size=3>
       <b>Label Encoding:</b>
Label Encoding is a technique used to convert categorical variables into numerical form. It assigns a unique integer value to each category, thereby creating an ordered relationship between the values. For example, if you have three categories: "red," "green," and "blue," label encoding may assign the values 0, 1, and 2, respectively.
   </font>
</p>

In [7]:
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

<p style="text-align: justify; text-justify: inter-word;">
   <font size=3>
       <b>One-Hot Encoding</b>, on the other hand, is a technique used to convert categorical variables into a binary vector representation. It creates new binary columns for each category, where a value of 1 represents the presence of that category and 0 represents the absence. Each category is treated as a separate feature. For example, using One-Hot Encoding, the three categories "red," "green," and "blue" would be represented as [1, 0, 0], [0, 1, 0], and [0, 0, 1], respectively.
   </font>
</p>


In [8]:
# https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html
ct = ColumnTransformer(transformers=[("encoder", 
                                      OneHotEncoder(), 
                                      [1] # Column number
                                     )
                                    ],
                       remainder="passthrough")
X = ct.fit_transform(X)

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [10]:
# Spliting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [11]:
# Applying feature scaling, to bring values at same scale
sc = StandardScaler()
sc_X_train = sc.fit(X_train)
X_train = sc_X_train.transform(X_train)
X_test = sc_X_train.transform(X_test)

### Model Building

<b>NOTE</b>: In Tensorflow2.0, keras is intergrated into tensorflow. 

<p style="text-align: justify; text-justify: inter-word;">
   <font size=3>
       The <code>tf.keras.models.Sequential()</code> class is a high-level API provided by TensorFlow for creating and defining neural network models in a sequential manner. It allows you to build models by stacking layers one after another, making it convenient for most deep learning tasks.
   </font>
</p>


In [12]:
ann = tf.keras.models.Sequential()

<p style="text-align: justify; text-justify: inter-word;">
   <font size=3>
       Usaully input treated as the first layer of neural network. Input layer are automatically added at the time of
       fitting.<br><br>
       In a neural network, a dense layer, also known as a fully connected layer, is a type of layer where each neuron or 
       node in the layer is connected to every neuron in the previous layer and every neuron in the next layer. The
       term "dense" refers to the fact that every node in the layer is connected to every node in the adjacent layers.
   </font>
</p>

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

<p style="text-align: justify; text-justify: inter-word;">
   <font size=3>
       Adding the second layer.
   </font>
</p>

In [14]:
ann.add(tf.keras.layers.Dense(units=6, activation="relu"))

<p style="text-align: justify; text-justify: inter-word;">
   <font size=3>
       Creating output layer. Since we have binary classes. We are using only one neuron
       in the output layer. If you have multiple classes, then you should use number of 
       neuron which match the count of your classes. For example, if you have cat, dog,
       cow classes then you will have 3 neurons in the output layer.
       Usually in output layer we use <i>sigmoid</i> or <i>softmax</i> as the activation
       function.
   </font>
</p>

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [16]:
# Compiling the ANN
ann.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [19]:
# Training the ANN on the training data set
ann.fit(X_train, y_train, batch_size=32, epochs=25)

Epoch 1/25
250/250 [==============================] - 6s 6ms/step - loss: 0.6926 - accuracy: 0.6233
Epoch 2/25
250/250 [==============================] - 1s 5ms/step - loss: 0.4997 - accuracy: 0.7933
Epoch 3/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4601 - accuracy: 0.7960
Epoch 4/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4413 - accuracy: 0.7975
Epoch 5/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4276 - accuracy: 0.8025
Epoch 6/25
250/250 [==============================] - 1s 3ms/step - loss: 0.4144 - accuracy: 0.8165
Epoch 7/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3982 - accuracy: 0.8263
Epoch 8/25
250/250 [==============================] - 1s 3ms/step - loss: 0.3818 - accuracy: 0.8364
Epoch 9/25
250/250 [==============================] - 1s 6ms/step - loss: 0.3690 - accuracy: 0.8451
Epoch 10/25
250/250 [==============================] - 1s 5ms/step - loss: 0.3613 - accuracy: 0.8503

In [20]:
ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (32, 6)                   78        
                                                                 
 dense_1 (Dense)             (32, 6)                   42        
                                                                 
 dense_2 (Dense)             (32, 1)                   7         
                                                                 
Total params: 127
Trainable params: 127
Non-trainable params: 0
_________________________________________________________________


In [21]:
sample = np.array([1.0, 0.0, 0.0, 600, 1, 40, 2, 60000, 2, 1, 1, 50000]).reshape(1, -1)

In [22]:
print(sample)

[[1.e+00 0.e+00 0.e+00 6.e+02 1.e+00 4.e+01 2.e+00 6.e+04 2.e+00 1.e+00
  1.e+00 5.e+04]]


In [23]:
sample_data = sc_X_train.transform(sample)
sample_data

array([[ 0.98560362, -0.5698444 , -0.57369368, -0.52111599,  0.91601335,
         0.10961719, -1.031415  , -0.2569057 ,  0.8095029 ,  0.64259497,
         0.9687384 , -0.87203322]])

In [24]:
print(ann.predict(sample_data))
# Checking by setting some threshold limit
print(ann.predict(sample_data) > 0.5)

1/1 [==============================] - 0s 211ms/step
[[0.04185354]]
1/1 [==============================] - 0s 47ms/step
[[False]]


In [25]:
# testing model using test data
y_pred = ann.predict(X_test)
y_pred = y_pred > 0.5
print(np.concatenate((
                      y_pred.reshape(len(y_pred), 1), 
                      y_test.reshape(len(y_test), 1)
                     ), 1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [26]:
# Evaluation Metrices
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1511   84]
 [ 196  209]]


0.86